# Text-to-Video Generation

**CCAI-413: Deep Learning project.**



------------------------------------------------------------------


# About 
We developed the model **text-to-video-ms-1.7b** which is the text-to-video generation model and improve it to support the Arabic language

Baseline model link: https://huggingface.co/damo-vilab/text-to-video-ms-1.7b/tree/main

# install the resources

In [1]:
!pip install diffusers transformers accelerate torch
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# import libraries

In [2]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from moviepy.editor import VideoFileClip
from googletrans import Translator
from requests.exceptions import ReadTimeout

# initialize the baseline code

In [3]:
# create the Translator
translator = Translator()

# handle ReadTimeout exception
def retry(func, max_retries=3, delay=1):
    for i in range(max_retries):
        try:
            return func()
        except ReadTimeout:
            print(f"Request timed out. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Exceeded maximum number of retries.")

# Creates DiffusionPipeline class and loads a pre-trained model 
# with specified parameters
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", 
                                         torch_dtype=torch.float16,
                                         variant="fp16")

# Configures the model's scheduler using the DPMSolverMultistepScheduler
# class with the configuration
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Enables offloading of model
pipe.enable_model_cpu_offload()



Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

# Text-to-Video Generation

In [4]:
#take the text from the user
text = input("Enter the text ")

#translate the input text from Arabic to English
# using Google Translate API.
translation = None
while translation is None:
    try:
        translation = translator.translate(text, src='ar', dest='en')
    except AttributeError:
        translation = retry(lambda: translator.translate(text, src='ar',
                                                         dest='en'))

# Extracts the text
prompt = translation.text

# Generates a sequence of video frames 
# The 'num_inference_steps': the number of inference steps
# to use during video generation.
video_frames = pipe(prompt, num_inference_steps=100).frames

#Exports the generated video frames to a video file
video_path = export_to_video(video_frames)

#Loads the video file into a VideoFileClip object
clip = VideoFileClip(video_path)

#Displays the video
clip.ipython_display(width=300)


Enter the text كلب يرقص


  0%|          | 0/100 [00:00<?, ?it/s]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
